# Gas

In [ ]:
import sys,os
sys.path.append(os.path.realpath(r'..\..\..\src\main\python'))
from mll import nb_logging_init
nb_logging_init()

import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from mll.LSTMPredictor import LSTMPredictor

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
tf.random.set_seed(13)

import mll.QuandlFeed as qf
df = qf.get_data({'CHRIS/CME_' + k: k for k in ['CL1', 'CL2', 'CL3', 'CL4', 'CL5']}, 'Last').dropna()
df.plot(subplots=True)

In [ ]:
TRAIN_SPLIT = int( len(df)*.7 )
past_history = 720
future_target = 72
predictor = LSTMPredictor(df, 'CL3', TRAIN_SPLIT, past_history, future_target)

In [ ]:
train_data_multi = predictor.train_data_multi #todo
multi_step_model = predictor.multi_step_model
multi_step_history = predictor.multi_step_history

In [ ]:
STEP=1
def create_time_steps(length):
  return list(range(-length, 0))

def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

for x, y in predictor.validation_dataset.take(3):
  multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])    